## Scraping IMDB reviews for movies with posters available on the OMBd API tool

In [107]:
import time
import csv
import os
import random
import re
import bs4 as bs
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

from urllib.request import FancyURLopener
from random import choice
from bs4 import SoupStrainer, BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome("/usr/local/bin/chromedriver", chrome_options=chrome_options)

user_agents = [
    "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
    "Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16",
    "Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246",
]


class MyOpener(FancyURLopener, object):
    version = choice(user_agents)

myopener = MyOpener()

In [118]:
file_list = []

for file in os.listdir("data/images/"):
    if file.endswith(".png"):
        file_list.append(str(file))
        
file_df = pd.DataFrame({'file_extension': file_list})
file_df.to_csv('file_extensions.csv', sep=',', index=False)

In [15]:
id_list = []

for file in os.listdir("data/images/"):
    if file.endswith('.png'):
        id_list.append(str(file).strip('.png'))

In [22]:
url_list = []

for i in range(len(id_list)):
    url_list.append('https://www.imdb.com/title/' + id_list[i])

In [ ]:
name_list = []
all_reviews = []
review_count = []

## Counter variable for debugging
count = 0
for x in range(len(url_list)):
    starter_url = url_list[x]
    page = myopener.open(starter_url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    
    time.sleep(random.randint(1, 5))
    
    ## Error handling for movies with no reviews
    try:
        user_reviews = soup.find(href='reviews?ref_=tt_ov_rt').get_text()
        
        review_text = []
        page = myopener.open(starter_url + '/reviews?ref_=tt_ov_rt')
        html = page.read().decode('utf-8')
        soup = BeautifulSoup(html, "html.parser")
        for text in soup.select('.show-more__control'):
            review_text.append(text.get_text())
        time.sleep(random.randint(1, 5))
        for text in soup.select('.parent'):
            name_list.append(text.get_text().split('\n\n')[1].split(' \n')[0])
        all_reviews.extend([review_text])
        review_count.append(int(user_reviews.strip(' user').replace(',', '')))
        
        count = count+1
    except AttributeError: 
        continue